# An end-to-end workflow to efficiently compress and deploy DNN classifiers on SoC/FPGA.

### Includes
- **config.py** file contains the definitions that the user should modify before the training and compression 
- **compressionStart.py** includes the functions used in the prcess

In [ ]:
import sys
sys.path.append("src")

from config import dataset_config, teacher_config, student_config

from src.compressionStart import *

##### `startDNNTrainingAndCompression()`: End-to-End Pipeline

This function orchestrates the full training and compression pipeline, including:

- **Dataset loading**  
  Automatically loads and preprocesses the selected dataset (1D signals or 2D images).

- **Teacher model optimization and training**  
  Performs hyperparameter tuning and training of the teacher model (if enabled via config).

- **Student model optimization (Bayesian Optimization)**  
  Tunes the student architecture based on the selected compression strategy:
  - **Quantization**
  - **Pruning**
  - **Knowledge Distillation**

- **Student model training**  
  Trains the student with the chosen compression techniques.

- **Performance evaluation**  
  Reports accuracy and other metrics for both teacher and student models.

- **Model saving**  
  Stores the trained teacher and student models in the `models/` directory, with filenames indicating the strategy used.


In [ ]:
startDNNTrainingAndCompression()    